In [ ]:
!pip install llama-index==0.9.36
!pip install asyncpg==0.29.0
%pip install openai --upgrade
dbutils.library.restartPython()

In [ ]:
from llama_index.llms.azure_openai import AzureOpenAI
from llama_index.embeddings.azure_openai import AzureOpenAIEmbedding
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader, ServiceContext
import logging
import sys
from collections.abc import Iterator
from llama_index.core import ServiceContext, SimpleDirectoryReader, StorageContext
from llama_index.core import VectorStoreIndex
from llama_index.vector_stores.postgres import PGVectorStore
import textwrap
import openai
#from llama_index import download_loader
# customize textnode - purpose is to add id to each node
#from llama_index.schema import TextNode
# customize stages of querying  https://docs.llamaindex.ai/en/latest/understanding/querying/querying.html
from llama_index.core import get_response_synthesizer
from llama_index.core.indices.vector_store.retrievers.retriever import VectorIndexRetriever
from llama_index.core.query_engine import RetrieverQueryEngine
from llama_index.core.postprocessor import SimilarityPostprocessor
#from llama_index import StorageContext, load_index_from_storage

logging.basicConfig(
    stream=sys.stdout, level=logging.INFO
)  # logging.DEBUG for more verbose output
logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))

In [ ]:
# Download query engine from Azure Blob Storage Container
from azure.storage.blob import BlobServiceClient, BlobClient, ContainerClient

# Your storage account connection string
connection_string = "DefaultEndpointsProtocol=https;AccountName=finessetestblobstorage;AccountKey=;EndpointSuffix=core.windows.net"

# The name of your container
container_name = "llamaindex-v1"

# The name of the virtual folder you want to list files from
folder_name = "index"

# Initialize the BlobServiceClient
blob_service_client = BlobServiceClient.from_connection_string(connection_string)

# Get the container client
container_client = blob_service_client.get_container_client(container_name)

# List all blobs in the specified folder
blobs_list = container_client.list_blobs(name_starts_with=folder_name)

# List all blobs in the container (at the root)
blobs_list = container_client.list_blobs()

for blob in blobs_list:
    print("Blob name: " + blob.name)
    blob_name = blob.name
    blob_client = blob_service_client.get_blob_client(container=container_name, blob=blob_name)
    # Download the blob to a local file
    download_file_path = "./index/" + blob_name
    with open(download_file_path, "wb") as download_file:
        download_file.write(blob_client.download_blob().readall())




In [ ]:
# Load query engine from local folder, 'index'
from llama_index.core import StorageContext, load_index_from_storage

# rebuild storage context
storage_context = StorageContext.from_defaults(persist_dir="./index")

# load index
index = load_index_from_storage(storage_context)

# configure retriever for debugging and retrieving metadata 
retriever = VectorIndexRetriever(
    index=index,
    similarity_top_k=15, # get top k documents.  15 is arbitrary
)

# configure response synthesizer
response_synthesizer = get_response_synthesizer()

# assemble query engine
query_engine = RetrieverQueryEngine(
    retriever=retriever,
    response_synthesizer=response_synthesizer,
    node_postprocessors=[SimilarityPostprocessor(similarity_cutoff=0.7)],
)

In [ ]:
# Example of query execute
response = query_engine.query("How do I import a cat from France to Canada?")
print(response)

In [ ]:
# get top k result into a list, in order of match score
top_k_result = []
for i in range(15): # arbitrary 15 because similarity_top_k=15 in this example
    top_k_result.append(response.source_nodes[i])
    
# get content
response.source_nodes[0].get_content()
# get embedding
response.source_nodes[0].embedding
# get score
response.source_nodes[0].get_score()
# get customized metadata. In this example, this retrieves chunk_id
response.source_nodes[0].metadata